In [2]:
# Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
# Handle warnings
import warnings
from warnings import filterwarnings
filterwarnings("ignore")

DATA COLLECTING

In [5]:
# Import dataset
data = pd.read_excel(r"/Users/jmc/GitHub/PredictAirplaneFare/predict-fares-ariplanes-ticket/dataset/dataset.xlsx")

DATA CLEANING

In [10]:
data.shape

(10683, 11)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


As we can see, almost all variables have object Dtype.
We'll have to address this with some preprocessing since ML works so much better with numerical features.

In [11]:
data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

Handling missing values

In [12]:
data[data['Route'].isnull()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


Since there is only one row with missing values, I'll delete this row. the impact is going to be minimum to the dataset.

In [13]:
data.dropna(inplace=True)

Data Preprocessing

In [15]:
df = data.copy()

In [18]:
def change_into_datetime(col):
    df[col] = pd.to_datetime(data[col])

In [21]:
for feature in ['Date_of_Journey', 'Arrival_Time','Dep_Time']:
    change_into_datetime(feature)

In [24]:
# Extract day, month, year of journey
df['Journey_Day'] = df['Date_of_Journey'].dt.day
df['Journey_Month'] = df['Date_of_Journey'].dt.month
df['Journey_Year'] = df['Date_of_Journey'].dt.year

In [28]:
def extract_hour_min(df, col):
    df[col+"_hour"] = df[col].dt.hour
    df[col+"_minute"] = df[col].dt.minute

In [32]:
extract_hour_min(df,'Arrival_Time')
extract_hour_min(df, 'Dep_Time')

In [34]:
col_to_drop = ['Arrival_Time', 'Dep_Time', 'Date_of_Journey']

In [35]:
df.drop(col_to_drop, axis=1, inplace=True)